<a href="https://colab.research.google.com/github/rukmals/ColabProject/blob/master/android.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
df=pd.read_csv('permission.csv')

In [5]:
df.head()

,android,android.app.cts.permission.TEST_GRANTED,android.intent.category.MASTER_CLEAR.permission.C2D_MESSAGE,android.os.cts.permission.TEST_GRANTED,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_ALL_EXTERNAL_STORAGE,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_COARSE_LOCATION,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,android.permission.ACCESS_DOWNLOAD_MANAGER,android.permission.ACCESS_DOWNLOAD_MANAGER_ADVANCED,android.permission.ACCESS_DRM_CERTIFICATES,android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_FM_RADIO,android.permission.ACCESS_INPUT_FLINGER,android.permission.ACCESS_KEYGUARD_SECURE_STORAGE,android.permission.ACCESS_LOCATION_EXTRA_COMMANDS,android.permission.ACCESS_MOCK_LOCATION,android.permission.ACCESS_MTP,android.permission.ACCESS_NETWORK_CONDITIONS,android.permission.ACCESS_NETWORK_STATE,android.permission.ACCESS_NOTIFICATIONS,android.permission.ACCESS_PDB_STATE,android.permission.ACCESS_SURFACE_FLINGER,android.permission.ACCESS_WIFI_STATE,android.permission.ACCESS_WIMAX_STATE,android.permission.ACCOUNT_MANAGER,android.permission.ALLOW_ANY_CODEC_FOR_PLAYBACK,android.permission.ASEC_ACCESS,android.permission.ASEC_CREATE,android.permission.ASEC_DESTROY,android.permission.ASEC_MOUNT_UNMOUNT,android.permission.ASEC_RENAME,android.permission.AUTHENTICATE_ACCOUNTS,android.permission.BACKUP,android.permission.BATTERY_STATS,android.permission.BIND_ACCESSIBILITY_SERVICE,android.permission.BIND_APPWIDGET,...,com.android.cts.permissionWithSignature,com.android.email.permission.ACCESS_PROVIDER,com.android.email.permission.READ_ATTACHMENT,com.android.frameworks.coretests.DANGEROUS,com.android.frameworks.coretests.NORMAL,com.android.frameworks.coretests.SIGNATURE,com.android.frameworks.coretests.keysets_permdef.keyset_perm,com.android.frameworks.coretests.permission.TEST_DENIED,com.android.frameworks.coretests.permission.TEST_GRANTED,com.android.gallery3d.filtershow.permission.READ,com.android.gallery3d.filtershow.permission.WRITE,com.android.gallery3d.permission.GALLERY_PROVIDER,com.android.launcher.permission.INSTALL_SHORTCUT,com.android.launcher.permission.PRELOAD_WORKSPACE,com.android.launcher.permission.READ_SETTINGS,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.android.launcher.permission.WRITE_SETTINGS,com.android.launcher3.permission.READ_SETTINGS,com.android.launcher3.permission.RECEIVE_FIRST_LOAD_BROADCAST,com.android.launcher3.permission.RECEIVE_LAUNCH_BROADCASTS,com.android.launcher3.permission.WRITE_SETTINGS,com.android.permission.WHITELIST_BLUETOOTH_DEVICE,com.android.printspooler.permission.ACCESS_ALL_PRINT_JOBS,com.android.providers.tv.permission.ACCESS_ALL_EPG_DATA,com.android.providers.tv.permission.ACCESS_WATCHED_PROGRAMS,com.android.providers.tv.permission.READ_EPG_DATA,com.android.providers.tv.permission.WRITE_EPG_DATA,com.android.smspush.WAPPUSH_MANAGER_BIND,com.android.voicemail.permission.ADD_VOICEMAIL,com.android.voicemail.permission.READ_VOICEMAIL,com.android.voicemail.permission.WRITE_VOICEMAIL,com.foo.mypermission,com.foo.mypermission2,org.chromium.chrome.shell.permission.C2D_MESSAGE,org.chromium.chrome.shell.permission.DEBUG,org.chromium.chrome.shell.permission.SANDBOX,org.chromium.chromecast.shell.permission.SANDBOX,org.chromium.content_shell.permission.SANDBOX,test_permission,type
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0

In [0]:
df['type'].describe()

count    398.000000
mean       0.500000
std        0.500629
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: type, dtype: float64

In [0]:
import tensorflow as tf

In [0]:
target = df.pop('type')

In [0]:
df=df.drop(['android','test_permission'],axis=1)

In [9]:
df.head()

,android.app.cts.permission.TEST_GRANTED,android.intent.category.MASTER_CLEAR.permission.C2D_MESSAGE,android.os.cts.permission.TEST_GRANTED,android.permission.ACCESS_ALL_DOWNLOADS,android.permission.ACCESS_ALL_EXTERNAL_STORAGE,android.permission.ACCESS_BLUETOOTH_SHARE,android.permission.ACCESS_CACHE_FILESYSTEM,android.permission.ACCESS_CHECKIN_PROPERTIES,android.permission.ACCESS_COARSE_LOCATION,android.permission.ACCESS_CONTENT_PROVIDERS_EXTERNALLY,android.permission.ACCESS_DOWNLOAD_MANAGER,android.permission.ACCESS_DOWNLOAD_MANAGER_ADVANCED,android.permission.ACCESS_DRM_CERTIFICATES,android.permission.ACCESS_FINE_LOCATION,android.permission.ACCESS_FM_RADIO,android.permission.ACCESS_INPUT_FLINGER,android.permission.ACCESS_KEYGUARD_SECURE_STORAGE,android.permission.ACCESS_LOCATION_EXTRA_COMMANDS,android.permission.ACCESS_MOCK_LOCATION,android.permission.ACCESS_MTP,android.permission.ACCESS_NETWORK_CONDITIONS,android.permission.ACCESS_NETWORK_STATE,android.permission.ACCESS_NOTIFICATIONS,android.permission.ACCESS_PDB_STATE,android.permission.ACCESS_SURFACE_FLINGER,android.permission.ACCESS_WIFI_STATE,android.permission.ACCESS_WIMAX_STATE,android.permission.ACCOUNT_MANAGER,android.permission.ALLOW_ANY_CODEC_FOR_PLAYBACK,android.permission.ASEC_ACCESS,android.permission.ASEC_CREATE,android.permission.ASEC_DESTROY,android.permission.ASEC_MOUNT_UNMOUNT,android.permission.ASEC_RENAME,android.permission.AUTHENTICATE_ACCOUNTS,android.permission.BACKUP,android.permission.BATTERY_STATS,android.permission.BIND_ACCESSIBILITY_SERVICE,android.permission.BIND_APPWIDGET,android.permission.BIND_CARRIER_MESSAGING_SERVICE,...,com.android.cts.permissionNormal,com.android.cts.permissionNotUsedWithSignature,com.android.cts.permissionWithSignature,com.android.email.permission.ACCESS_PROVIDER,com.android.email.permission.READ_ATTACHMENT,com.android.frameworks.coretests.DANGEROUS,com.android.frameworks.coretests.NORMAL,com.android.frameworks.coretests.SIGNATURE,com.android.frameworks.coretests.keysets_permdef.keyset_perm,com.android.frameworks.coretests.permission.TEST_DENIED,com.android.frameworks.coretests.permission.TEST_GRANTED,com.android.gallery3d.filtershow.permission.READ,com.android.gallery3d.filtershow.permission.WRITE,com.android.gallery3d.permission.GALLERY_PROVIDER,com.android.launcher.permission.INSTALL_SHORTCUT,com.android.launcher.permission.PRELOAD_WORKSPACE,com.android.launcher.permission.READ_SETTINGS,com.android.launcher.permission.UNINSTALL_SHORTCUT,com.android.launcher.permission.WRITE_SETTINGS,com.android.launcher3.permission.READ_SETTINGS,com.android.launcher3.permission.RECEIVE_FIRST_LOAD_BROADCAST,com.android.launcher3.permission.RECEIVE_LAUNCH_BROADCASTS,com.android.launcher3.permission.WRITE_SETTINGS,com.android.permission.WHITELIST_BLUETOOTH_DEVICE,com.android.printspooler.permission.ACCESS_ALL_PRINT_JOBS,com.android.providers.tv.permission.ACCESS_ALL_EPG_DATA,com.android.providers.tv.permission.ACCESS_WATCHED_PROGRAMS,com.android.providers.tv.permission.READ_EPG_DATA,com.android.providers.tv.permission.WRITE_EPG_DATA,com.android.smspush.WAPPUSH_MANAGER_BIND,com.android.voicemail.permission.ADD_VOICEMAIL,com.android.voicemail.permission.READ_VOICEMAIL,com.android.voicemail.permission.WRITE_VOICEMAIL,com.foo.mypermission,com.foo.mypermission2,org.chromium.chrome.shell.permission.C2D_MESSAGE,org.chromium.chrome.shell.permission.DEBUG,org.chromium.chrome.shell.permission.SANDBOX,org.chromium.chromecast.shell.permission.SANDBOX,org.chromium.content_shell.permission.SANDBOX
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42)

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [0]:
model = get_compiled_model()


In [11]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=30,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_test, y_test))

Epoch 1/30
5/5 [==============================] - 0s 56ms/step - loss: 0.6923 - accuracy: 0.5189 - val_loss: 0.6872 - val_accuracy: 0.4250
Epoch 2/30
5/5 [==============================] - 0s 26ms/step - loss: 0.6793 - accuracy: 0.5189 - val_loss: 0.6743 - val_accuracy: 0.4250
Epoch 3/30
5/5 [==============================] - 0s 32ms/step - loss: 0.6675 - accuracy: 0.5189 - val_loss: 0.6625 - val_accuracy: 0.4250
Epoch 4/30
5/5 [==============================] - 0s 26ms/step - loss: 0.6553 - accuracy: 0.5252 - val_loss: 0.6509 - val_accuracy: 0.4250
Epoch 5/30
5/5 [==============================] - 0s 30ms/step - loss: 0.6436 - accuracy: 0.5440 - val_loss: 0.6400 - val_accuracy: 0.4250
Epoch 6/30
5/5 [==============================] - 0s 30ms/step - loss: 0.6328 - accuracy: 0.5440 - val_loss: 0.6287 - val_accuracy: 0.4250
Epoch 7/30
5/5 [==============================] - 0s 28ms/step - loss: 0.6209 - accuracy: 0.5849 - val_loss: 0.6167 - val_accuracy: 0.4875
Epoch 8/30
5/5 [===========

In [23]:
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

ANDROID_SAVED_MODEL = "saved_models/android"
tf.saved_model.save(model, ANDROID_SAVED_MODEL)

INFO:tensorflow:Assets written to: saved_models/android/assets


In [16]:
android_model = hub.load(ANDROID_SAVED_MODEL)
print(android_model)



<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f555a5aab70>


In [20]:
!mkdir "tflite_models"

mkdir: cannot create directory ‘tflite_models’: File exists


In [0]:
TFLITE_MODEL = "tflite_models/android.tflite"
TFLITE_QUANT_MODEL = "tflite_models/android_quant.tflite"
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

TFLite models and their sizes:
total 24K
-rw-r--r-- 1 root root 5.7K May 28 03:29 android_quant.tflite
-rw-r--r-- 1 root root  16K May 28 03:29 android.tflite


In [28]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(tflite_model)


# Convert the model to quantized version with post-training quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)

print("TFLite models and their sizes:")
!ls "tflite_models" -lh

TFLite models and their sizes:
total 24K
-rw-r--r-- 1 root root 5.2K May 28 13:41 android_quant.tflite
-rw-r--r-- 1 root root  15K May 28 13:41 android.tflite
